<a href="https://colab.research.google.com/github/Living-with-machines/zooniverse-analysis-workshop/blob/main/process-zooniverse-data-for-deposit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Zooniverse data for analysis and deposit

## This Notebook will combine Zooniverse classification and subject files into a single CSV with redacted usernames and identifying information

The default Zooniverse data export contains nested JSON data in a plain text CSV file. This can be hard to process. It also contains potentially identifying information such as usernames and IP addresses. This Notebook is designed for Zooniverse project owners who want to access classification (annotation) data and subject records in a single CSV file that doesn't contain personal data.

This Notebook reads Zooniverse classification and subject CSV files and:

1. Combines the information from:  
    (a) the **classifications** file, which contains all the annotations users have created for a given workflow's subjects, with  
    (b) the **subjects** file, which contains the information about the subjects (items) annotated in specific tasks.

2. It transforms difficult-to-read information in JSON format into spreadsheet columns for ease of use.

3. It removes any Personally Identifiable Information (PII), such as usernames, IP addresses and user IDs, replacing them with a 'user_name_redacted' ID.

4. It produces a .csv file that can be opened in any spreadsheet tool (Microsoft Excel, Apple Numbers, etc.) for further processing.

### Caveats
This code is not optimised for very large files. It has been tested on files with c 8000 classifications.

The Notebook can be used with workflow-specific classification files or the general classification file which contains annotations from all tasks.

The Notebook does not calculate consensus or volunteer agreement on classifications.

This Notebook is based on one originally created by Giorgia Tolfo, Kalle Westerling and Mia Ridge for a British Library workshop for the Living with Machines project in May 2022. It was updated in November 2023 by Mia Ridge and Nilo Pedrazzini.

## Getting set up

In the next cell, you will specify where the data that you want to process is located. These should be two paths on your drive, which can be _relative_ (to where this notebook is located) or _absolute_ (a specified full path to each file).

#### Example 1: Sample data

If you do not have any downloaded data, but want to use the sample data that we have provided (located in the same folder as this notebook), you should put these two _relative_ paths in the next cell:

```py
classifications_file = "sample_data/test_classifications.csv"
subjects_file = "sample_data/test_subjects.csv"
```

Pandas is a very flexible Python package, which can also accept valid URLs, so you could also write:

```py
classifications_file = "https://raw.githubusercontent.com/Living-with-machines/zooniverse-analysis-workshop/main/sample_data/test_classifications.csv"
subjects_file = "https://raw.githubusercontent.com/Living-with-machines/zooniverse-analysis-workshop/main/sample_data/test_subjects.csv"
```

#### Example 2: Downloaded data in Downloads folder

If you have downloaded the files as `classifications.csv` and `subjects.csv` on a Mac and they are located in your `Downloads` folder, in the next cell you would put two _absolute_ paths:

```py
classifications_file = "/Users/<your-username>/Downloads/classifications.csv"
subjects_file = "/Users/<your-username>/Downloads/subjects.csv"
```


#### Example 3: Data in your Google Drive

```py
# Code originally from https://github.com/kingsdigitallab/lwm-davizct
# Update the paths to match the location of files on your Google Drive
# Running this code will trigger a dialogue to allow access to your Google Drive
ipython = get_ipython()
IN_COLAB = "google.colab" in str(ipython)

if IN_COLAB:
    from google.colab import drive

    drive.mount("/content/drive")
    data_path = "drive/My Drive/Zooniverse_exports"
    sources_path = "drive/My Drive/Zooniverse_exports" # not needed in this notebook?
    subjects_path = "drive/My Drive/Zooniverse_exports" # not needed in this notebook?

```

## Now it is your turn!

**Fill in the file locations in this cell:**

In [ ]:
# Running this code will trigger a dialogue to allow access to your Google Drive
ipython = get_ipython()
IN_COLAB = "google.colab" in str(ipython)

if IN_COLAB:
    from google.colab import drive

    drive.mount("/content/drive", force_remount=True)
    data_path = "drive/My Drive/Zooniverse_exports/"

Mounted at /content/drive


In [ ]:
classifications_file = data_path + "bicycle-or-motorcycle-classifications.csv"
subjects_file = data_path + "subjects.csv"

---

## Setting up our definitions

**Now, we are ready to get started working with our data.**

First, we need to import the packages that we are going to use in the script below. Most of them are built-in to Python.

In [ ]:
from collections import ChainMap
import hashlib
import json

import pandas as pd

## Read in our main data

Next, it's time to read in the main CSV data as a pandas DataFrame, using the handy method `.read_csv()`.

If this stage fails, go back up and check the path to your data files

In [ ]:
df = pd.read_csv(classifications_file)

Pandas has a handy method to look at some of the rows of data that we have just imported into a dataframe. When you call the `.head(num)` method, replace `num` with the number of rows in the frame that you want to see.

We run it below to see what our imported data looks like unprocessed:

In [ ]:
df.head(3)

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
0,472682035,miaridge,1926730.0,da69c883ec25160c9ea7,23672,Bicycle or motorcycle?,3.1,2023-03-07 18:34:38 UTC,NaN,NaN,"{""source"":""api"",""session"":""7d8205d7a4d2fb54876...","[{""task"":""T0"",""task_label"":""## How did the wor...","{""85991045"":{""retired"":{""id"":113082478,""workfl...",85991045
1,472682242,miaridge,1926730.0,da69c883ec25160c9ea7,23672,Bicycle or motorcycle?,3.1,2023-03-07 18:35:15 UTC,NaN,NaN,"{""source"":""api"",""session"":""7d8205d7a4d2fb54876...","[{""task"":""T0"",""task_label"":""## How did the wor...","{""85990912"":{""retired"":{""id"":113080249,""workfl...",85990912
2,472833458,npedrazzini,2426416.0,7ae8040e256d95640f75,23672,Bicycle or motorcycle?,3.1,2023-03-08 09:10:00 UTC,NaN,NaN,"{""source"":""api"",""session"":""5e8ec73229377a60a07...","[{""task"":""T0"",""task_label"":""## How did the wor...","{""85991200"":{""retired"":{""id"":113083082,""workfl...",85991200


As we can see in our preview, the `classification_id` is a column, which is unique for each classification. It can thus be used as an "index" for the DataFrame, a nice way of querying the frame by individual IDs.

Here's how we can set the index on our `df`:

In [ ]:
df.set_index("classification_id", inplace=True)

Python and Pandas both 'succeed silently' - they don't give you an update if a command is successful. But did that command do what you expected? You can reality check the results by displaying the first rows using head(), or try the sample() method to see a random selection of rows.

In [ ]:
df.sample(3)

,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
classification_id,,,,,,,,,,,,,
473094185,databanana,1682891.0,ac683a04b3e74ab47765,23672,Bicycle or motorcycle?,3.1,2023-03-09 10:56:56 UTC,NaN,NaN,"{""source"":""api"",""session"":""3ebb20d93388e1b1d82...","[{""task"":""T0"",""task_label"":""## How did the wor...","{""85991627"":{""retired"":{""id"":113084035,""workfl...",85991627
473483944,BerylP,2554935.0,7b95f335ac2477505a76,23672,Bicycle or motorcycle?,3.1,2023-03-10 21:49:20 UTC,NaN,NaN,"{""source"":""api"",""session"":""9878f9a9140493e487e...","[{""task"":""T0"",""task_label"":""## How did the wor...","{""85991014"":{""retired"":{""id"":113084168,""workfl...",85991014
473287759,serea3,2436271.0,8e1e05f782c689587b4d,23672,Bicycle or motorcycle?,3.1,2023-03-10 03:23:19 UTC,NaN,NaN,"{""source"":""api"",""session"":""67c71cf64952403a4e3...","[{""task"":""T0"",""task_label"":""## How did the wor...","{""85991184"":{""retired"":{""id"":113083472,""workfl...",85991184


## Ensure correct encoding

Next, we will want to have a look at the encoding of each column.

In the "reality check" above, you can see that the columns `subject_data`, `annotations`, and `metadata` are structured as JSON (JavaScript Object Notation), which the DataFrame parser cannot interpret on its own, which is why we have to help it using the following parsing of those specific columns.

If you use Pandas' `.apply()` method (which can be applied on a column or on the entire DataFrame), you can help the software interpret the data in the columns. Here, we pass a function (`json.loads`, i.e. the `loads` function from Python's built-in `json` package) to each of the columns containing JSON data.

In [ ]:
df["metadata"] = df["metadata"].apply(json.loads)
df["annotations"] = df["annotations"].apply(json.loads)

Again, we might want to do a "reality check". To look at a specific part of a dataframe we can "slice it" by passing it a list (note the double `[[` and `]]` on each side of the selector) of the columns we want to look at:

In [ ]:
df[["annotations", "metadata"]]

,annotations,metadata
classification_id,,
472682035,"[{'task': 'T0', 'task_label': '## How did the ...","{'source': 'api', 'session': '7d8205d7a4d2fb54..."
472682242,"[{'task': 'T0', 'task_label': '## How did the ...","{'source': 'api', 'session': '7d8205d7a4d2fb54..."
472833458,"[{'task': 'T0', 'task_label': '## How did the ...","{'source': 'api', 'session': '5e8ec73229377a60..."
472833469,"[{'task': 'T0', 'task_label': '## How did the ...","{'source': 'api', 'session': '5e8ec73229377a60..."
472833485,"[{'task': 'T0', 'task_label': '## How did the ...","{'source': 'api', 'session': '5e8ec73229377a60..."
...,...,...
475507435,"[{'task': 'T0', 'task_label': '## How did the ...","{'source': 'api', 'session': '4d8d986200fd5a4c..."
475507451,"[{'task': 'T0', 'task_label': '## How did the ...","{'source': 'api', 'session': '4d8d986200fd5a4c..."
475507508,"[{'task': 'T0', 'task_label': '## How did the ...","{'source': 'api', 'session': '4d8d986200fd5a4c..."


Looks good!

In the following two sections, we will process the `metadata`, and `annotations` column respectively.

## Extract classification metadata (`metadata`)

Next, we want to do something similar to what we did with the subjects above to process the metadata for each classification.

Because the data in the `metadata` column has nested JSON data (that is, objects and lists that are wrapped inside each other), we want to use the particular `json_normalize` method.

Here's an example of what one `metadata` row looks like:

In [ ]:
df.metadata[df.metadata.head(1).index[0]]

{'source': 'api',
 'session': '7d8205d7a4d2fb54876f678f5672493cf02ed82a7d31854228ee466c640d0bd6',
 'viewport': {'width': 1081, 'height': 687},
 'started_at': '2023-03-07T18:30:44.874Z',
 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:107.0) Gecko/20100101 Firefox/107.0',
 'utc_offset': '0',
 'finished_at': '2023-03-07T18:34:38.586Z',
 'live_project': True,
 'interventions': {'opt_in': True, 'messageShown': False},
 'user_language': 'en',
 'user_group_ids': [],
 'subject_dimensions': [{'clientWidth': 310,
   'clientHeight': 500,
   'naturalWidth': 310,
   'naturalHeight': 500}],
 'subject_selection_state': {'retired': False,
  'selected_at': '2023-03-07T18:30:44.866Z',
  'already_seen': False,
  'selection_state': 'normal',
  'finished_workflow': False,
  'user_has_finished_workflow': False},
 'workflow_translation_id': '62778'}

Now, let's create a separate DataFrame from the normalised JSON data in the columns using the json_normalize method and use `.head()` to see the result:

In [ ]:
df_metadata = pd.json_normalize(df["metadata"])

df_metadata.head(3)

,source,session,started_at,user_agent,utc_offset,finished_at,live_project,user_language,user_group_ids,subject_dimensions,...,viewport.height,interventions.opt_in,interventions.messageShown,subject_selection_state.retired,subject_selection_state.selected_at,subject_selection_state.already_seen,subject_selection_state.selection_state,subject_selection_state.finished_workflow,subject_selection_state.user_has_finished_workflow,seen_before
0,api,7d8205d7a4d2fb54876f678f5672493cf02ed82a7d3185...,2023-03-07T18:30:44.874Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,0,2023-03-07T18:34:38.586Z,True,en,[],"[{'clientWidth': 310, 'clientHeight': 500, 'na...",...,687.0,True,False,False,2023-03-07T18:30:44.866Z,False,normal,False,False,NaN
1,api,7d8205d7a4d2fb54876f678f5672493cf02ed82a7d3185...,2023-03-07T18:34:41.585Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,0,2023-03-07T18:35:15.654Z,True,en,[],"[{'clientWidth': 322, 'clientHeight': 500, 'na...",...,687.0,True,False,False,2023-03-07T18:30:44.866Z,False,normal,False,False,NaN
2,api,5e8ec73229377a60a07a96fc497d02a5414b374ec3705a...,2023-03-08T09:09:10.078Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,0,2023-03-08T09:10:00.473Z,True,en,[],"[{'clientWidth': 133, 'clientHeight': 500, 'na...",...,886.0,True,False,False,2023-03-08T09:09:09.729Z,False,normal,False,False,NaN


Since we know that the shape of the classification metadata DataFrame (`df_metadata`) and the main classification DataFrame (`df`) are the same, we can apply the `set_index` method to the metadata to get the `classification_id` as index on the `df_metadata`:

In [ ]:
df_metadata.set_index(df.index, inplace=True)

Reality check time again — good practice, keep track of what you're doing:

In [ ]:
df_metadata.head(3)

,source,session,started_at,user_agent,utc_offset,finished_at,live_project,user_language,user_group_ids,subject_dimensions,...,viewport.height,interventions.opt_in,interventions.messageShown,subject_selection_state.retired,subject_selection_state.selected_at,subject_selection_state.already_seen,subject_selection_state.selection_state,subject_selection_state.finished_workflow,subject_selection_state.user_has_finished_workflow,seen_before
classification_id,,,,,,,,,,,,,,,,,,,,,
472682035,api,7d8205d7a4d2fb54876f678f5672493cf02ed82a7d3185...,2023-03-07T18:30:44.874Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,0,2023-03-07T18:34:38.586Z,True,en,[],"[{'clientWidth': 310, 'clientHeight': 500, 'na...",...,687.0,True,False,False,2023-03-07T18:30:44.866Z,False,normal,False,False,NaN
472682242,api,7d8205d7a4d2fb54876f678f5672493cf02ed82a7d3185...,2023-03-07T18:34:41.585Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,0,2023-03-07T18:35:15.654Z,True,en,[],"[{'clientWidth': 322, 'clientHeight': 500, 'na...",...,687.0,True,False,False,2023-03-07T18:30:44.866Z,False,normal,False,False,NaN
472833458,api,5e8ec73229377a60a07a96fc497d02a5414b374ec3705a...,2023-03-08T09:09:10.078Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,0,2023-03-08T09:10:00.473Z,True,en,[],"[{'clientWidth': 133, 'clientHeight': 500, 'na...",...,886.0,True,False,False,2023-03-08T09:09:09.729Z,False,normal,False,False,NaN


## Extract annotations (`annotations`)

The final information we want to extract and process from the main DataFrame are the annotations for each classification.

In order to do so, we must first preprocess the JSON data that we have available, extracting a list of only the information we want (the "annotation values"). We set up a function that processes each individual row, and then apply it to each row, using the `.apply` method, which we have used before:

In [ ]:
def extract_annotation_values(annotation_row):
    """
    Takes an annotation row, which contains a list of tasks with values in dictionary {task, task_label, value}
    and extracts the `task_label` and `value` for each `task` and returns them as a dictionary,
    for easy insertion into a DataFrame.
    """

    extracted_dictionaries = [{'{}_task_label'.format(task_data.get('task')): task_data.get('task_label'), '{}_task_value'.format(task_data.get('task')): ''.join(task_data.get('value'))} for task_data in annotation_row]

    return dict(ChainMap(*extracted_dictionaries))

df["annotations"] = df["annotations"].apply(extract_annotation_values)

Then, similarly to section 4 above, we loop through each row (of JSON data - contained in `json_data`) and extract the classification_id and annotations for each of them, which we then add on our new DataFrame `df_annotations`:

In [ ]:
df_annotations = pd.DataFrame()

for index, row in df.iterrows():
    s = pd.Series(row.annotations, name=index)
    df_annotations = pd.concat([df_annotations, s], axis=1)

df_annotations = df_annotations.T

Finally, we can use the `.head()` method again to check our results:

In [ ]:
df_annotations.head(3)

,T0
472682035,[## How did the word 'bike' change over time a...
472682242,[## How did the word 'bike' change over time a...
472833458,[## How did the word 'bike' change over time a...


## Extract Zooniverse subjects (`subject_data`)

Next, we want to read in the `subjects.csv` file above, so we know what files each of the classifications were done on.

In [ ]:
df_subjects = pd.read_csv(subjects_file)

Let's have a quick look at our data using `.sample()`:

In [ ]:
df_subjects.sample(3)

,project_id,workflow_id,subject_set_id,metadata,locations,classifications_count,retired_at,retirement_reason,created_at,updated_at
subject_id,,,,,,,,,,
82923090,9943,20921.0,109748,"{""!page"":""0008"",""!text"":""Highly-important Sale...","{""0"":""https://panoptes-uploads.zooniverse.org/...",4,2023-01-30 14:08:34 UTC,consensus,2022-11-23 12:43:04 UTC,2023-04-02 14:43:11 UTC
82933569,9943,20921.0,109762,"{""!page"":""0006"",""!text"":""Au alarming fire brok...","{""0"":""https://panoptes-uploads.zooniverse.org/...",3,NaN,NaN,2022-11-23 17:23:34 UTC,2023-03-31 23:16:39 UTC
37476961,9943,12194.0,77929,"{""!page"":""0003"",""!image"":""crop_0000151_1846121...","{""0"":""https://panoptes-uploads.zooniverse.org/...",0,NaN,NaN,2019-10-13 21:22:08 UTC,2022-11-09 19:25:45 UTC


As with the classifications, we can see from this preview that the `subject_id` is a column, which is unique for each subject. Similarly to what we did with the classifications above, we can turn it into an "index" for the DataFrame, a nice way of querying the frame by individual IDs. Again, here's how we'd do it:

In [ ]:
df_subjects.set_index("subject_id", inplace=True)

Now, let's see what happened:

In [ ]:
df_subjects.sample(3)

,project_id,workflow_id,subject_set_id,metadata,locations,classifications_count,retired_at,retirement_reason,created_at,updated_at
subject_id,,,,,,,,,,
52881587,9943,19348.0,89678,"{""!page"":""0004"",""!image"":""cropped_WO1_NRWC_187...","{""0"":""https://panoptes-uploads.zooniverse.org/...",3,2021-08-27 20:24:35 UTC,classification_count,2020-12-01 08:38:46 UTC,2022-11-09 19:42:39 UTC
85733097,9943,NaN,111674,"{"""":""1558"",""!page"":""5"",""!text"":""HEALTH COMMITT...","{""0"":""https://panoptes-uploads.zooniverse.org/...",0,NaN,NaN,2023-02-20 15:51:48 UTC,2023-03-31 18:09:54 UTC
73334530,9943,22513.0,105221,"{""!myid"":""0002597_18770811_0003_pa0003098"",""!p...","{""0"":""https://panoptes-uploads.zooniverse.org/...",4,2022-10-17 13:49:13 UTC,classification_count,2022-02-25 16:00:29 UTC,2022-11-09 16:54:14 UTC


As we can see from the preview, there are some quirky aspects of the `subjects.csv` file from Zooniverse. Two things we might want to address here is that the `metadata` column is formatted as embedded JSON data, and so is the `locations` column.

The `metadata` column corresponds to the data from any manifest file that you uploaded with your subject sets, so it often contains valuable information for us to have with our classifications.

The `locations` column contains the URL to all of the images that each subject links to. For our projects, we might just have one location, but not necessarily: subjects can contain multiple images!

For our purposes, we want to create a more readable list of locations and extract the metadata into its own columns. Let's go ahead and do that now.

We will start by making sure that both columns are formatted correctly, as we did with the classifications' metadata and annotations above:

In [ ]:
df_subjects["metadata"] = df_subjects["metadata"].apply(json.loads)
df_subjects["locations"] = df_subjects["locations"].apply(json.loads)

We will start with the `locations` column. Since the data is structured as a Python dictionary `{id: "URL"}`, we can get a list of all the URLs by using Python's `dict` type's built-in `.values()` method. Here, we put that method into a custom function, which takes any row from the `subjects` DataFrame, makes a list from its values, and then joins them together with a comma separation (see the `", ".join()` syntax):

In [ ]:
def locations_as_list(row):
    return ", ".join(list(row.values()))

df_subjects["locations_list"] = df_subjects["locations"].apply(locations_as_list)

Now, to save some space, we can remove the old `locations` columns:

In [ ]:
df_subjects = df_subjects.drop(["locations"], axis="columns")

Next, we will turn to the subjects' `metadata` column.

Similarly to the extraction of classifications metadata above, we will create a separate DataFrame from the normalised JSON data from the subjects column using the `json_normalize` method and then use `.head()` to see the result:

In [ ]:
df_subjects_metadata = pd.json_normalize(df_subjects["metadata"])

df_subjects_metadata.sample(3)

,!page,!image,!origin,attribution,!inferred_nlp,newspaper date,!crop_rectangle,newspaper place,newspaper title,!zooniverse_file_md5,...,!original_metadata.newspaper title,!original_metadata.!zooniverse_file_md5,!original_metadata.!zooniverse_file_stem,,!issue,!article_id,!textblock_id,newspaper_date,newspaper_place,newspaper_title
93267,8,0002609-18880526-art0094-pa0008070-accidents-0...,BritishNewspaperArchive,NaN,0002609,1888-05-26,"[4948, 6170, 8160, 9294]","Runcorn, Cheshire, England",Runcorn Examiner,55fd81c458c05d8706a242be0ce89bce,...,NaN,NaN,NaN,1315,18880526,art0094,pa0008070,NaN,NaN,NaN
45816,0003,crop_0002595_18750618_0003_engine_injured_1822...,NaN,Image from THE BRITISH LIBRARY BOARD.,0002595,1875-06-18,"[1822, 2972, 2392, 4753]","Blakeney, Gloucestershire, England",The Forest of Dean Examiner,4e19d35b055e37159c479daa2d4aafc5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95784,7,0002612-18740808-art0070-pa0007066-accidents-0...,BritishNewspaperArchive,NaN,0002612,1874-08-08,"[2422, 1683, 2982, 2621]","Stoke-on-Trent, Staffordshire, England",The Potteries Examiner,ede318d8406b41ff1c367e93286febc3,...,NaN,NaN,NaN,362,18740808,art0070,pa0007066,NaN,NaN,NaN


Again, since we know that the shape of the subjects' metadata DataFrame (`df_subjects_metadata`) and the main subject DataFrame (`df_subject`) are the same, we can apply the `set_index` method to the metadata to get the `subject_id` as index on the `df_subjects_metadata`.

After, we'll run the `.head()` reality check, to make sure all is well:

In [ ]:
df_subjects_metadata.set_index(df_subjects.index, inplace=True)

df_subjects_metadata.head(3)

,!page,!image,!origin,attribution,!inferred_nlp,newspaper date,!crop_rectangle,newspaper place,newspaper title,!zooniverse_file_md5,...,!original_metadata.newspaper title,!original_metadata.!zooniverse_file_md5,!original_metadata.!zooniverse_file_stem,,!issue,!article_id,!textblock_id,newspaper_date,newspaper_place,newspaper_title
subject_id,,,,,,,,,,,,,,,,,,,,,
36627626,0005,crop_0000064_18410319_0005_accident_1076_4276_...,BritishNewspaperArchive,Image © THE BRITISH LIBRARY BOARD. ALL RIGHTS ...,0000064,1841-03-19,"[1076, 4276, 1961, 6677]","Hull, Humberside, England",The Hull Packet,6b001b8fc3eea13df5738d139a0f8b29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36627627,0005,crop_0000064_18410319_0005_accident_1900_1038_...,BritishNewspaperArchive,Image © THE BRITISH LIBRARY BOARD. ALL RIGHTS ...,0000064,1841-03-19,"[1900, 1038, 2745, 1245]","Hull, Humberside, England",The Hull Packet,8a332a4b4a3fdc62ac24c87bec094cd7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36627628,0005,crop_0000064_18410319_0005_accident_1912_1564_...,BritishNewspaperArchive,Image © THE BRITISH LIBRARY BOARD. ALL RIGHTS ...,0000064,1841-03-19,"[1912, 1564, 2756, 1727]","Hull, Humberside, England",The Hull Packet,c2c1f956810b6deb9ddb3e4a8ac270f9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since we've now extracted the `metadata` column into a separate DataFrame, let's go ahead and remove the column from the original or "main" `df_subjects` DataFrame:

In [ ]:
df_subjects = df_subjects.drop("metadata", axis="columns")

A really great feature of DataFrames is that you can join them back together, as long as the `pandas` library has an easy way of matching the two DataFrames (or more!) together. A joined index, such as we have for `df_subjects` and `df_subjects_metadata` is a great way to do so.

In order to get one large `df_subjects` DataFrame that contains all the data for each subject, we can thus run the `.join` method:

In [ ]:
df_subjects = df_subjects.join(df_subjects_metadata)

Once again, we see that `pandas` succeeds silently, so let's do a reality check:

In [ ]:
df_subjects.head(3)

,project_id,workflow_id,subject_set_id,classifications_count,retired_at,retirement_reason,created_at,updated_at,locations_list,!page,...,!original_metadata.newspaper title,!original_metadata.!zooniverse_file_md5,!original_metadata.!zooniverse_file_stem,,!issue,!article_id,!textblock_id,newspaper_date,newspaper_place,newspaper_title
subject_id,,,,,,,,,,,,,,,,,,,,,
36627626,9943,12038.0,77830,0,NaN,NaN,2019-09-11 13:09:40 UTC,2022-11-09 17:53:37 UTC,https://panoptes-uploads.zooniverse.org/subjec...,0005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36627627,9943,12038.0,77830,0,NaN,NaN,2019-09-11 13:09:40 UTC,2022-11-09 17:53:46 UTC,https://panoptes-uploads.zooniverse.org/subjec...,0005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36627628,9943,12038.0,77830,0,NaN,NaN,2019-09-11 13:09:40 UTC,2022-11-09 17:51:18 UTC,https://panoptes-uploads.zooniverse.org/subjec...,0005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


That is one large (but handy) DataFrame!

## Joining all the data back together

In this final step, we want to join the data back together into a new main DataFrame, `df_final`, from which we will remove all the personal data so the dataset can be shared publicly, if we want.

To recap, here are the four DataFrames that we have created thus far:

#### Main DataFrame (`df`)

In [ ]:
df.head(1)

,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
classification_id,,,,,,,,,,,,,
472682035,miaridge,1926730.0,da69c883ec25160c9ea7,23672,Bicycle or motorcycle?,3.1,2023-03-07 18:34:38 UTC,NaN,NaN,"{'source': 'api', 'session': '7d8205d7a4d2fb54...",{'T0': ['## How did the word 'bike' change ove...,"{""85991045"":{""retired"":{""id"":113082478,""workfl...",85991045


#### Subjects DataFrame (`df_subjects`)

In [ ]:
df_subjects.head(1)

,project_id,workflow_id,subject_set_id,classifications_count,retired_at,retirement_reason,created_at,updated_at,locations_list,!page,...,!original_metadata.newspaper title,!original_metadata.!zooniverse_file_md5,!original_metadata.!zooniverse_file_stem,,!issue,!article_id,!textblock_id,newspaper_date,newspaper_place,newspaper_title
subject_id,,,,,,,,,,,,,,,,,,,,,
36627626,9943,12038.0,77830,0,NaN,NaN,2019-09-11 13:09:40 UTC,2022-11-09 17:53:37 UTC,https://panoptes-uploads.zooniverse.org/subjec...,0005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Metadata DataFrame (`df_metadata`)

In [ ]:
df_metadata.head(1)

,source,session,started_at,user_agent,utc_offset,finished_at,live_project,user_language,user_group_ids,subject_dimensions,...,viewport.height,interventions.opt_in,interventions.messageShown,subject_selection_state.retired,subject_selection_state.selected_at,subject_selection_state.already_seen,subject_selection_state.selection_state,subject_selection_state.finished_workflow,subject_selection_state.user_has_finished_workflow,seen_before
classification_id,,,,,,,,,,,,,,,,,,,,,
472682035,api,7d8205d7a4d2fb54876f678f5672493cf02ed82a7d3185...,2023-03-07T18:30:44.874Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,0,2023-03-07T18:34:38.586Z,True,en,[],"[{'clientWidth': 310, 'clientHeight': 500, 'na...",...,687.0,True,False,False,2023-03-07T18:30:44.866Z,False,normal,False,False,NaN


#### Annotations DataFrame (`df_annotations`)

In [ ]:
df_annotations.head(1)

,T0
472682035,[## How did the word 'bike' change over time a...


### Drop existing data

As you can see from the summary above, of the main DataFrame (`df`), it now doubles the information in `df_subjects`, `df_metadata`, and `df_annotations`. So before we do anything else, we will drop the columns that we have now processed from `df` to reduce the size of `df`:

In [ ]:
df = df.drop(["subject_data", "metadata", "annotations"], axis="columns")

### Redact personal information

Next, we want to ensure that we have anonymised the user data.

Here, we have created our own function, `redact_username`, which uses a cryptographic hash method from Python's built-in `hashlib` library called `sha256`. You can read more about [the algorithm's history on Wikipedia](https://en.wikipedia.org/wiki/SHA-2) if you are interested.

In [ ]:
def redact_username(row):
    return hashlib.sha256(str(row).encode()).hexdigest() if not pd.isna(row) else None

df["user_name_redacted"] = df["user_name"].apply(redact_username)

We might also be interested in knowing whether a user was logged in or not when they annotated the subject. In order to preserve that information, we create another custom function, `user_was_logged_in`, which uses the information from the `user_name` column (which has a value like `not-logged-in-98ff168ef257e2fd9d4a` if the user was not logged in) to extract a `True` or `False` value (also called a "boolean"):

In [ ]:
def user_was_logged_in(row):
    return "not-logged-in" not in row if not pd.isna(row) else False

df["user_logged_in"] = df["user_name"].apply(user_was_logged_in)

Finally, we want to drop columns that contain personal identifying information - usernames, IDs and IP addresses - from the main DataFrame:

In [ ]:
df = df.drop(["user_id", "user_name", "user_ip"], axis="columns")

### Join all the data

Now, it's time to join all the DataFrames into one new `df_final` frame.

Adding the metadata and the annotations on the main `df` is an easy thing, since all of them share index:

In [ ]:
df_final = df.join(df_metadata).join(df_annotations)

Let's do a quick reality check to make sure that all the data is there:

In [ ]:
df_final.head(1)

,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,subject_ids,user_name_redacted,user_logged_in,source,...,interventions.opt_in,interventions.messageShown,subject_selection_state.retired,subject_selection_state.selected_at,subject_selection_state.already_seen,subject_selection_state.selection_state,subject_selection_state.finished_workflow,subject_selection_state.user_has_finished_workflow,seen_before,T0
classification_id,,,,,,,,,,,,,,,,,,,,,
472682035,23672,Bicycle or motorcycle?,3.1,2023-03-07 18:34:38 UTC,NaN,NaN,85991045,130db9e15df39e75f7ac8c70512f28f9a894d15063cbc1...,True,api,...,True,False,False,2023-03-07T18:30:44.866Z,False,normal,False,False,NaN,[## How did the word 'bike' change over time a...


So far, so good. But now we want to join our subjects on the `df_final` table as well. It's not entirely as easy as the previous multiple join task that we just did here.

This time, because we're not using data points that are uniquely connected, one-to-one (one subject, in fact, will have many classifications), we will end up with duplicated information in the table. In order to join the main DataFrame with the subject DataFrame `df_subjects`, we need to make one intermediary step: We need to make both index columns available in each of the DataFrames.

_Why do we need to make `df_final`'s index a column?_ We will _merge_ two DataFrames, which is a function that will discard both indices of the merging DataFrames.

In [ ]:
df_subjects["subject_ids"] = df_subjects.index
df_final["classification_id"] = df_final.index

Before we run the merge, we also want to make sure that the two matching columns on both DataFrames are of the same (correct) type—otherwise Pandas will have trouble finding the matching subject row for each classification.

For that, we use the built-in method on each DataFrame, `.astype()` which "casts" a column as a type:

In [ ]:
df_subjects["subject_ids"] = df_subjects["subject_ids"].astype(int)
df_final["subject_ids"] = df_final["subject_ids"].astype(int)

Now we should be able to use the `merge` function from Pandas central library to join the two on the column that is shared between the two DataFrames (in our case `subject_ids`):

In [ ]:
df_merged = pd.merge(df_final, df_subjects, on="subject_ids", how="left", suffixes=("_classification", "_subject"))

Finally, we can make the `classification_id` column an index again, on our final DataFrame:

In [ ]:
df_merged = df_merged.set_index("classification_id")

One last reality check, before we're ready to export!

This time, however, we set the max column to be displayed to `None` because we're interested in seeing all the columns. Then we use our now familiar `.head()` method:

In [ ]:
pd.set_option('display.max_columns', None)
df_merged.head(3)

,workflow_id_classification,workflow_name,workflow_version,created_at_classification,gold_standard,expert,subject_ids,user_name_redacted,user_logged_in,source,session,started_at,user_agent,utc_offset,finished_at,live_project,user_language,user_group_ids,subject_dimensions,workflow_translation_id,viewport.width,viewport.height,interventions.opt_in,interventions.messageShown,subject_selection_state.retired,subject_selection_state.selected_at,subject_selection_state.already_seen,subject_selection_state.selection_state,subject_selection_state.finished_workflow,subject_selection_state.user_has_finished_workflow,seen_before,T0,project_id,workflow_id_subject,subject_set_id,classifications_count,retired_at,retirement_reason,created_at_subject,updated_at,locations_list,!page,!image,!origin,attribution,!inferred_nlp,newspaper date,!crop_rectangle,newspaper place,newspaper title,!zooniverse_file_md5,!zooniverse_file_stem,!text,!text_source,!myid,origin,!original_id,!original_metadata.!myid,!original_metadata.!page,!original_metadata.!image,!original_metadata.origin,!original_metadata.attribution,!original_metadata.!text_source,!original_metadata.!inferred_nlp,!original_metadata.newspaper date,!original_metadata.!crop_rectangle,!original_metadata.newspaper place,!original_metadata.newspaper title,!original_metadata.!zooniverse_file_md5,!original_metadata.!zooniverse_file_stem,,!issue,!article_id,!textblock_id,newspaper_date,newspaper_place,newspaper_title
classification_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
472682035,23672,Bicycle or motorcycle?,3.1,2023-03-07 18:34:38 UTC,NaN,NaN,85991045,130db9e15df39e75f7ac8c70512f28f9a894d15063cbc1...,True,api,7d8205d7a4d2fb54876f678f5672493cf02ed82a7d3185...,2023-03-07T18:30:44.874Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,0,2023-03-07T18:34:38.586Z,True,en,[],"[{'clientWidth': 310, 'clientHeight': 500, 'na...",62778,1081.0,687.0,True,False,False,2023-03-07T18:30:44.866Z,False,normal,False,False,NaN,[## How did the word 'bike' change over time a...,9943,23672.0,111974,3,2023-03-13 17:19:33 UTC,classification_count,2023-03-07 07:45:06 UTC,2023-04-02 15:11:43 UTC,https://panoptes-uploads.zooniverse.org/subjec...,0002,0003040-18961003-0002-bike_pa0002091.png,lwmnewspapers,NaN,0003040,NaN,"[3009, 407, 3573, 1311]",NaN,NaN,1edbb0e29ae0329c0d22bf8cf84efcfb,NaN,A CYCLE TOUR. [Continual] Oh ! the boletereas ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1003,0002,pa0002091,1896-10-03,"Birkenhead, Merseyside, England",The Birkenhead News and Wirral General Adverti...
472682242,23672,Bicycle or motorcycle?,3.1,2023-03-07 18:35:15 UTC,NaN,NaN,85990912,130db9e15df39e75f7ac8c70512f28f9a894d15063cbc1...,True,api,7d8205d7a4d2fb54876f678f5672493cf02ed82a7d3185...,2023-03-07T18:34:41.585Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,0,2023-03-07T18:35:15.654Z,True,en,[],"[{'clientWidth': 322, 'clientHeight': 500, 'na...",62778,1081.0,687.0,True,False,False,2023-03-07T18:30:44.866Z,False,normal,False,False,NaN,[## How did the word 'bike' change over time a...,9943,23672.0,111974,3,2023-03-09 07:09:19 UTC,classification_count,2023-03-07 07:43:53 UTC,2023-04-02 15:14:37 UTC,https://panoptes-uploads.zooniverse.org/subjec...,0004,0003039-18990919-0004-bike_pa0004006.png,lwmnewspapers,NaN,0003039,NaN,"[172, 678, 567, 1291]",NaN,NaN,4a7b52d720a1d7bd86760ddb07f1d12d,NaN,too large for bin. PAUtia Perkins. left alarm ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0919,0004,pa0004006,1899-09-19,"Hartlepool, Cleveland, England",The Northern Guardian.
472833458,23672,Bicycle or motorcycle?,3.1,2023-03-08 09:10:00 UTC,NaN,NaN,85991200,a00fd0167e3595bdaad31871b7b7c187ebd393597ec460...,True,api,5e8ec73229377a60a07a96fc497d02a5414b374ec3705a...,2023-03-08T09:09:10.078Z,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,0,2023-03-08T09:10:00.473Z,True,en,[],"[{'clientWidth': 133, 'clientHeight': 500, 'na...",62778,1284.0,88

## Export!

In this final step, we can export the file to whatever format that we'd like to use.

If you have used a workflow-specific classifications file, you will probably want to include the workflow name or ID in the export filename.

In the case below, we opt for `.csv`, which is a common, open file format that can be easily opened in Microsoft Excel for further processing. It can also be imported into visualisation software, such as Power BI, or websites like ObservableHQ. On Colab, the file will be stored in a temporary directory. Once you've run the code below, click the 'folder' icon to view and download it:

In [ ]:
df_merged.to_csv("combined-bicycle-or-motorcycle-classifications.csv")